Hi guys,<br>
Welcome back to the pandas essentials, in this lecture we are going to talk about
# Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:<br>
  
* <code><b>merge():</b></code> connects rows in DataFrames based on one or more keys. (*This will be familiar to SQL or other relational databases users, as it implements database join operations*).
* <code><b>concat():</b></code> concatenate or "stacks" together objects along an axis.


&#9989; If you don't know SQL, don't worry, the concepts of merging are presented with very simple examples so that you can follow the steps. Although, our focus here is not to learn SQL, we only want to go through the widely used and few very important <code>**inner**</code> and <code>**outer**</code> joining operations for data wrangling. <br>
If you have questions, please ask and we are more than happy to help! <br>
&#9989; Important thing you should know: Merging operation may give <code>**NaN**</code> in the output and they needs to be treated according to the circumstances/requirements during data analysis.  

Let's discuss these methods with examples.

### Database-Style DataFrame joins
Merge or join operations combine datasets by linking rows using one or more keys. These operations are central to relational databases (e.g., SQL-based).<br>

In [ ]:
import pandas as pd
import numpy as np

We need data to work with, let's create two DataFrames, df1 and df2.

In [ ]:
df1 = pd.DataFrame({'key': ['a', 'b', 'c', 'd', 'e'],
                    'A1': range(5),
                    'B1':range(5,10)})

df2 = pd.DataFrame({'key': ['a', 'b', 'c'],
                    'A2': range(3),
                    'B2':range(3,6)})

Always good to see how our data look like!

In [ ]:
df1

,key,A1,B1
0,a,0,5
1,b,1,6
2,c,2,7
3,d,3,8
4,e,4,9


In [ ]:
df2

,key,A2,B2
0,a,0,3
1,b,1,4
2,c,2,5


We have created dataframes, they look great. <br>
Before we move on, let's explore <code><b>'merge()'</b></code> method first. <br>
We can type pd.merge and press shift+tab in the Jupyter notebook to see the documentation.<br>
There are several parameters that we can pass to the merge method, the most important ones are <code><b>'how'</b></code> and <code><b>'on'</b></code>, that we will discuss here. <br>

* <code><b>'how'</b></code> tells the <code><b>'merge()'</b></code>, what type of joining operation needs to be done, it could be <code><b>'inner','outer','left','right'</b></code>. Default value of <code><b>'how'</b></code> is <code><b>'inner</b></code>, if nothing is provided.  

* <code><b>'on'</b></code> tells the field name to join on, which could be a label or a list. <br>

## <code>merge()</code>

Let's overview <code><b>'how'</b></code> and <code><b>'on'</b></code> parameters in <code><b>'merge()'</b></code>.<br>

<code><b>how:{'inner','outer','left','right'}</b></code><br>
* <code><b>'inner':</b></code> use intersection of keys from both frames, similar to a SQL inner join.
* <code><b>'outer':</b></code> use union of keys from both frames, similar to a SQL full outer join.
* <code><b>'left':</b></code> use only keys from left frame, similar to a SQL left outer join.
* <code><b>'right':</b></code> use only keys from right frame, similar to a SQL right outer join.

<code><b>on:label or list</b></code>
* Field names to join on.
* Must be found in both DataFrames.

&#9758; d, e did not appear in the merged output, <code><b>'inner'</b></code> returns the intersection of key columns only!

### <code>how = 'Outer'</code>

* The key column in the result will be the union of <code>df1['key']</code> and <code>df2['key']</code>, means, all the keys found in both tables.<br>

&#9758; I am using <code>print</code> to out put resultant along with the original dataframes <code>df1</code>, <code>df2</code> to do the comparisons.

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   key     5 non-null      object
 1   A1      5 non-null      int64 
 2   B1      5 non-null      int64 
dtypes: int64(2), object(1)
memory usage: 252.0+ bytes


### how = 'inner'

In [ ]:
pd.merge(df1, df2, how = 'inner', on='key')[['key', 'A1', 'A2']]

,key,A1,A2
0,a,0,0
1,b,1,1
2,c,2,2


&#9758; <code>NaN</code> in A2, B2 columns for d, e indexes. Its Union operation and A2, B2 values does not exist in df2 for indexes d, e!

### <code>how ='left'</code>
Use only key column of the left dataframe, similar to a SQL left outer join.

&#9758; <code>NaN</code> for indexes d, e in A2, B2, as indexes d, e don't exist in <code>df2['key']</code>.

In [ ]:
type(np.nan)

float

In [ ]:
pd.merge(df1, df2, how = 'left', on='key')

,key,A1,B1,A2,B2
0,a,0,5,0.0,3.0
1,b,1,6,1.0,4.0
2,c,2,7,2.0,5.0
3,d,3,8,NaN,NaN
4,e,4,9,NaN,NaN


### how = 'right'
Use only key column of the right dataframe, similar to a SQL right outer join.

In [ ]:
pd.merge(df1, df2, how = 'right',on='key')

,key,A1,B1,A2,B2
0,a,0,5,0,3
1,b,1,6,1,4
2,c,2,7,2,5


### how = 'outer'

In [ ]:
pd.merge(df1, df2, how = 'outer',on='key')

,key,A1,B1,A2,B2
0,a,0,5,0.0,3.0
1,b,1,6,1.0,4.0
2,c,2,7,2.0,5.0
3,d,3,8,NaN,NaN
4,e,4,9,NaN,NaN


#### Merging example with two key (key1, key2) columns -- little complicated!
Let's create two data frames such that each have two key columns, <code>key1 & key2</code>.

In [ ]:
left = pd.DataFrame({'key1': ['a', 'a', 'b', 'c'],
                     'key2': ['a', 'b', 'a', 'b'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})

right = pd.DataFrame({'key1': ['a', 'b', 'b', 'c'],
                      'key2': ['a', 'b', 'a', 'a'],
                      'C': ['C0', 'C1', 'C2', 'C3'],
                      'D': ['D0', 'D1', 'D2', 'D3']})

<code><b>'inner'</b></code> is intersection, only the key pair present in both dataframes will appear in the resultant

In [ ]:
left

,key1,key2,A,B
0,a,a,A0,B0
1,a,b,A1,B1
2,b,a,A2,B2
3,c,b,A3,B3


In [ ]:
right

,key1,key2,C,D
0,a,a,C0,D0
1,b,b,C1,D1
2,b,a,C2,D2
3,c,a,C3,D3


In [ ]:
pd.merge(left, right, how = 'inner', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,a,a,A0,B0,C0,D0
1,b,a,A2,B2,C2,D2


As we know, <code><b>'outer'</b></code> is union, all key pair present in both dataframes will appear in the resultant.

In [ ]:
pd.merge(left, right, how='outer', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,a,a,A0,B0,C0,D0
1,a,b,A1,B1,NaN,NaN
2,b,a,A2,B2,C2,D2
3,c,b,A3,B3,NaN,NaN
4,b,b,NaN,NaN,C1,D1
5,c,a,NaN,NaN,C3,D3


For <code><b>'left'</b></code> join, the key pair in left will be used only

In [ ]:
pd.merge(left, right, how='left', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,a,a,A0,B0,C0,D0
1,a,b,A1,B1,NaN,NaN
2,b,a,A2,B2,C2,D2
3,c,b,A3,B3,NaN,NaN


For <code><b>'right'</b></code> join, the key pair in right will be used only

In [ ]:
pd.merge(left, right, how='right', on=['key1', 'key2'])

,key1,key2,A,B,C,D
0,a,a,A0,B0,C0,D0
1,b,b,NaN,NaN,C1,D1
2,b,a,A2,B2,C2,D2
3,c,a,NaN,NaN,C3,D3


## Concatenation

Concatenation is interchangeably referred as binding, or stacking as well. This operation basically glues together DataFrames. <br>

&#9758; It's important to remember that dimensions should match along the axis, we are concatenating on. <br>

We can use <code><b>pd.concat</b></code> and pass in a list of DataFrames to concatenate together.<br>
Let's create two simple dataframes, with the given indexes, to understand concatenation.

In [ ]:
df1 = pd.DataFrame({'A': ['A0', 'A1', 'A2', 'A3'],
                        'B': ['B0', 'B1', 'B2', 'B3'],
                        'C': ['C0', 'C1', 'C2', 'C3'],
                        'D': ['D0', 'D1', 'D2', 'D3']},
                        index=[0, 1, 2, 3])

In [ ]:
df2 = pd.DataFrame({'A': ['A4', 'A5', 'A6', 'A7'],
                        'B': ['B4', 'B5', 'B6', 'B7'],
                        'C': ['C4', 'C5', 'C6', 'C7'],
                        'D': ['D4', 'D5', 'D6', 'D7']},
                         index=[4,5,6,7])

In [ ]:
df1

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3


In [ ]:
df2

,A,B,C,D
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


In [ ]:
pd.concat([df1,df2]) # default axis is 0/'index' to concatenate along

,A,B,C,D
0,A0,B0,C0,D0
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4
5,A5,B5,C5,D5
6,A6,B6,C6,D6
7,A7,B7,C7,D7


# Good Job
This section was little tricky, practice can make the difference. Let's overview.<br>
We are make great progress. <br>
**Is not it cool!**